# IEEE-CIS Fraud Detection
Решение этой задачи приведено в рамках прохождения курса MADMO (Сбербанк).<br>
Автор Ерошкин И.С.

## Постановка задачи
По данным о транзакциях и идентификационным данным необходимо оценить вероятность мошенничества при проведении операций с пластиковыми картами.<br>
Описание соревнования приведено по ссылке: https://www.kaggle.com/c/ieee-fraud-detection

# Данные

**Данные содержат:**
* Transaction Table
* Identity Table

Описание таблиц приведено по ссылке:
https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203

## Transaction Table<br>
1. TransactionDT: timedelta from a given reference datetime (not an actual timestamp)<br>
1. TransactionAMT: transaction payment amount in USD<br>
1. ProductCD: product code, the product for each transaction<br>
1. card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.<br>
1. addr: address<br>
1. dist: distance<br>
1. P_ and (R__) emaildomain: purchaser and recipient email domain<br>
1. C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.<br>
1. D1-D15: timedelta, such as days between previous transaction, etc.<br>
1. M1-M9: match, such as names on card and address, etc.<br>
1. Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.<br>
<br>
Categorical Features:
1. ProductCD
1. card1 - card6
1. addr1, addr2
1. Pemaildomain Remaildomain
1. M1 - M9

## Identity Table

Categorical Features:
1. DeviceType
1. DeviceInfo
1. id12 - id38

# Результат, метрика оценки, формат результата
* Для каждой транзакции TransactionID необходимо оценить вероятность мошеничества isFraud.
* Метрика оценки - ROC AUC.
* Результирующий файл должен иметь формат:<br>
TransactionID,isFraud<br>
3663549,0.5<br>
3663550,0.5<br>
3663551,0.5<br>
и т.д.<br>



При подготовке использованы фрагменты из:<br>
* https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt
* https://www.kaggle.com/artgor/eda-and-models
* https://www.kaggle.com/polmast/demin-av-eee-cis

# Part_0: Ипморт и загрузка данных

In [ ]:
# !pip install -U vega_datasets notebook vega

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error, roc_auc_score
# pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# import eli5
# import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# alt.renderers.enable('notebook')

# %env JOBLIB_TEMP_FOLDER=/tmp


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression

In [ ]:
def get_unique(df):
    uv = []
    cols = []
    for col in list(df.columns):
        uv.append(df[col].unique().shape[0])
        cols.append(col)
    df_uv = pd.DataFrame({'predictor': cols, 'uv': uv}  )
    return df_uv



## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# import pathlib
# pathlib.Path().absolute()
print(os.listdir("../input/"))

In [ ]:
# загрузка датасетов
train_identity = pd.read_csv(r'../input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv')
test_identity = pd.read_csv('../input/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv(f'../input/ieee-fraud-detection/test_transaction.csv')
sub = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv') 

# Part_1: Исследование данных

In [ ]:
print('train_identity.shape: ', train_identity.shape)
print('train_transaction.shape: ', train_transaction.shape)
print('test_identity.shape: ', test_identity.shape)
print('test_transaction.shape: ', test_transaction.shape)

In [ ]:
train_transaction.head(2)

In [ ]:
train_identity.head(2)

In [ ]:
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
# Поскольку загруженные данные занимают много оперативной памяти, то удалим из памяти первично загруженные датасеты:
del train_identity, train_transaction, test_identity, test_transaction
# дополнительнопринудительно вызывем сборщик мусора, который почистит память (примерно 3 гб):
gc.collect()

### **Целевая переменная несбалансирована**

In [ ]:
y_dsb = train.groupby('isFraud')['isFraud'].count()
print('Доля хитов целевой переменной: {:.1%}'.format(y_dsb[1]/(y_dsb[1]+y_dsb[0])))

sns.distplot(train.isFraud,kde=False);

In [ ]:
train_cols = pd.DataFrame({'type': train.dtypes}).reset_index()
train_cols['uv'] = get_unique(train).uv
train_cols.head()

In [ ]:
null_sum = train.isnull().sum()
null_cnt = train.isnull().count()

In [ ]:
train_cols.index = train_cols['index']
train_cols.drop(['index'], axis=1,inplace=True)
train_cols['num_vals'] = null_cnt
train_cols['num_nans'] = null_sum
train_cols['num_nans_perc'] = null_sum / null_cnt
train_cols.head()

In [ ]:
train_cols.sort_values(by='num_nans_perc', ascending=False)

#### Выделим предикторы у которых пропусков менее 50%

In [ ]:
train_cols['cat'] = np.nan
train_cols.loc[train_cols.num_nans_perc>0.5, 'cat'] = 'del'

#### Размечаем числовые и категориальные величины

In [ ]:
train_cols[(train_cols.cat!='del')].groupby('type')['type'].count()

In [ ]:
train_cols.loc[(train_cols.cat!='del')&(train_cols.type!='object'), 'cat'] = 'num'
train_cols.loc[(train_cols.cat!='del')&(train_cols.type=='object'), 'cat'] = 'cat'

train_cols.groupby('cat')['cat'].count()

#### Проверим, что нет колонок с одним уникльным значением

In [ ]:
train_cols.sort_values(by='uv', ascending=True).head(20)

In [ ]:
train_cols[train_cols.cat=='cat']

In [ ]:
for col in list(train_cols[train_cols.cat=='cat'].index):
    print(col)
    plt.subplots(1,2,figsize=(15,5))
    plt.subplot(1,2,1)
    sns.countplot(train[col]);
    plt.subplot(1,2,2)
    sns.barplot(x=col, y='isFraud', data=train);
    plt.show()

In [ ]:
# убираем из предикторов колонку 'M1'
train_cols.loc[train_cols.index=='M1', 'cat']='del'

In [ ]:
train_cols.loc[train_cols.cat=='num'].sort_values('uv')

In [ ]:
train_cols.loc[train_cols.cat=='num']

In [ ]:
# i=0
# for col in train_cols.loc[train_cols.cat=='num'].index:
#     print(i)
#     print(col)
#     train.hist(col)
#     i = i + 1
#     plt.show()

# Part_2: Генерация и отбор предикторов

### Генерация предикторов_01

In [ ]:
# преобразуем TransactionDT в стандартную дату и создадим новую фичу Date
import datetime

genesis = datetime.datetime.strptime('2019-01-01', '%Y-%m-%d')
train['Date'] = train['TransactionDT'].apply(lambda x : genesis+datetime.timedelta(seconds=x))

In [ ]:
# создадим новые фичи:
train['Weekdays'] = train['Date'].dt.dayofweek
train['Days'] = train['Date'].dt.day
train['Hours'] = train['Date'].dt.hour

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

g = sns.barplot(train.Weekdays, train.isFraud, ax=ax[0])
ax[0].set_title('Fraud Charges by Weekdays')
plt.setp(g.get_xticklabels(), visible=False)

g = sns.barplot(train.Days, train.isFraud, ax=ax[1])
ax[1].set_title('Fraud Charges by Days')
plt.setp(g.get_xticklabels(), visible=False)

g = sns.barplot(train.Hours, train.isFraud, ax=ax[2])
ax[2].set_title('Fraud Charges by Hours')
plt.setp(g.get_xticklabels(), visible=False)

plt.show()

In [ ]:
test['Date'] = test['TransactionDT'].apply(lambda x : genesis+datetime.timedelta(seconds=x))
# создадим новые фичи:
test['Weekdays'] = test['Date'].dt.dayofweek
test['Days'] = test['Date'].dt.day
test['Hours'] = test['Date'].dt.hour

In [ ]:
train.drop(['Date'], axis=1,inplace=True)
test.drop(['Date'], axis=1,inplace=True)

### Отбор предикторов

In [ ]:
train_cols[train_cols.cat.isnull()].count()

In [ ]:
train_cols.head(2)

In [ ]:
train_cols.loc[train_cols.index=='TransactionDT', :]

In [ ]:
train_cols.loc[train_cols.index=='isFraud', 'cat']='target'
train_cols.loc[train_cols.index=='TransactionDT', 'cat']='datetime'

In [ ]:
categorical_columns = list(train_cols.loc[train_cols.cat=='cat',:].index)
numerical_columns = list(train_cols.loc[train_cols.cat=='num',:].index)
categorical_columns

In [ ]:
numerical_columns

In [ ]:
%%time
X = train[categorical_columns + numerical_columns]
y = train['isFraud']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y)

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, categorical_columns),
     ('num', numerical_pipe, numerical_columns)])

rf = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', RandomForestClassifier())
])

rf.fit(X_train, y_train)

In [ ]:
print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))

In [ ]:
ohe = (rf.named_steps['preprocess']
         .named_transformers_['cat']
         .named_steps['onehot'])
feature_names = ohe.get_feature_names(input_features=categorical_columns)
feature_names = np.r_[feature_names, numerical_columns]

tree_feature_importances = (
    rf.named_steps['classifier'].feature_importances_)
sorted_idx = tree_feature_importances.argsort()

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots(figsize=(10,160))
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(feature_names[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances")
fig.tight_layout()
plt.show()

In [ ]:
# %%time
# result = permutation_importance(rf, X_test, y_test, n_repeats=5)
# sorted_idx = result.importances_mean.argsort()

### Сохранение и чтение результата perm_importance

In [ ]:
# df_tmp = pd.DataFrame()
# for i in range(result['importances'].shape[1]):
#     col_nm = 'col_'+str(i)
# #     print(col_nm)
#     df_tmp[col_nm]=result['importances'][:,i]
# df_tmp.head(3) 
# df_tmp.to_csv('permut_importances.csv', index=False)

In [ ]:
df_tmp2 = pd.read_csv('../input/permut-importances02/permut_importances_02.csv')
tmp3 = np.array(df_tmp2)
result2={'importances': tmp3, 'importances_mean': tmp3.mean(axis=1), 'importances_std': tmp3.std(axis=1)}

### Анализ результата perm_importance

In [ ]:
result = result2
sorted_idx = result['importances_mean'].argsort()

fig, ax = plt.subplots(figsize=(10,160))
ax.boxplot(result['importances'][sorted_idx].T,
           vert=False, labels=X_test.columns[sorted_idx])
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()

In [ ]:
df_importances = pd.DataFrame({'predictor': X_test.columns[sorted_idx], 'importance': result['importances_mean'][sorted_idx]})
df_importances['importances_abs'] = df_importances[['importance']].apply(abs)
df_importances = df_importances.sort_values(by='importances_abs',ascending=True)
df_importances.iloc[-5:,:]

In [ ]:
plt.figure(figsize=(10,160))
plt.barh(df_importances.predictor, df_importances.importances_abs);

In [ ]:
df_importances.iloc[-5:,:]

In [ ]:
train_cols2=train_cols.reset_index()
train_cols2=train_cols2.rename(columns={'index': 'predictor1'})
train_cols2 = train_cols2.merge(df_importances, how='left', left_on='predictor1', right_on='predictor')
train_cols2 = train_cols2.sort_values(by='importances_abs',ascending=False).reset_index(drop=True)
train_cols2['importances_abs'] = train_cols2[['importances_abs']].apply(lambda x: x/x.max()*100)
train_cols2[100:160]

In [ ]:
col_cat_top = list(train_cols2[:30].loc[train_cols2.cat=='cat','predictor'])
col_num_top = list(train_cols2[:30].loc[train_cols2.cat=='num','predictor'])

In [ ]:
for col in col_cat_top:
    print(col)
    plt.subplots(1,2,figsize=(15,5))
    plt.subplot(1,2,1)
    sns.countplot(train[col]);
    plt.subplot(1,2,2)
    sns.barplot(x=col, y='isFraud', data=train);
    plt.show()

In [ ]:
for col in col_num_top:
    print(col)
#     plt.subplots(1,1,figsize=(15,5))
#     plt.subplot(1,2,1)
#     sns.countplot(train[col]);
#     plt.subplot(1,2,2)
#     sns.barplot(x=col, y='isFraud', data=train);
    sns.distplot(train[col],kde=False);
    plt.show()

In [ ]:
train_cols2.groupby('cat')['cat'].count()

In [ ]:
train_cols2.loc[train_cols2.predictor=='TransactionID','cat']='del'
train_cols2.loc[train_cols2.predictor=='TransactionID',:]

train_cols2.loc[train_cols2.importances_abs<=3,'cat']='del2'
train_cols2.groupby('cat')['cat'].count()

In [ ]:
train_cols = train_cols2
train_cols.index = train_cols['predictor1']
train_cols.head(2)

### Генерация предикторов_02

In [ ]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')


train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('std')

In [ ]:
numerical_columns_02 = ['TransactionAmt_to_mean_card1','TransactionAmt_to_mean_card4','TransactionAmt_to_std_card1','TransactionAmt_to_std_card4','D15_to_mean_card1','D15_to_mean_card4','D15_to_std_card1','D15_to_std_card4','D15_to_mean_addr1','D15_to_mean_addr2','D15_to_std_addr1','D15_to_std_addr2']

In [ ]:
# by https://www.kaggle.com/dimartinot
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
train = clean_inf_nan(train)
test = clean_inf_nan(test )

# Part_3: Моделирование

In [ ]:
# df_tmp = pd.DataFrame({'col': categorical_columns + numerical_columns})
# df_tmp.groupby('col')['col'].count().sort_values( ascending=False)

### Part_3: Моделирование: 01_RandomForest

In [ ]:
categorical_columns = list(train_cols.loc[train_cols.cat=='cat',:].index)
numerical_columns = list(train_cols.loc[train_cols.cat=='num',:].index) + numerical_columns_02

X = train[categorical_columns + numerical_columns]
y = train['isFraud']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y)

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, categorical_columns),
     ('num', numerical_pipe, numerical_columns)])

pipe = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', RandomForestClassifier())
])

In [ ]:
param_grid = {
    'classifier__max_depth': [2, 5, 20]
}

grid_search_01 = GridSearchCV(pipe, param_grid, scoring='roc_auc',cv=3)

In [ ]:
%%time
grid_search_01.fit(X_train,y_train)

In [ ]:
grid_search_01.best_score_ 

In [ ]:
y_pred = grid_search_01.predict(X_test)
y_pred_proba = grid_search_01.predict_proba(X_test)[:,1]
auc_res_01 = roc_auc_score(y_test,y_pred_proba)
auc_res_01

In [ ]:
grid_search_01.best_estimator_ 

### Part_3: Моделирование: 02_XGBoost

In [ ]:
categorical_columns = list(train_cols.loc[train_cols.cat=='cat',:].index)
numerical_columns = list(train_cols.loc[train_cols.cat=='num',:].index) + numerical_columns_02

X = train[categorical_columns + numerical_columns]
y = train['isFraud']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y)

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, categorical_columns),
     ('num', numerical_pipe, numerical_columns)])

pipe = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', xgb.XGBClassifier())
])

In [ ]:
param_grid = {
    'classifier__max_depth': [20,30]
}

grid_search_02 = GridSearchCV(pipe, param_grid, scoring='roc_auc',cv=3)

In [ ]:
%%time
grid_search_02.fit(X_train,y_train)

In [ ]:
grid_search_02.best_score_

In [ ]:
y_pred = grid_search_02.predict(X_test)
y_pred_proba = grid_search_02.predict_proba(X_test)[:,1]
auc_res_02 = roc_auc_score(y_test,y_pred_proba)
auc_res_02

### Part_3: Моделирование: 03_LogisticRegression

In [ ]:
categorical_columns = list(train_cols.loc[train_cols.cat=='cat',:].index)
numerical_columns = list(train_cols.loc[train_cols.cat=='num',:].index) + numerical_columns_02

sc = StandardScaler()

X = train[categorical_columns + numerical_columns]
y = train['isFraud']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y)

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, categorical_columns),
     ('num', numerical_pipe, numerical_columns)])

pipe = Pipeline([
    ('preprocess', preprocessing),
    ('sc', sc),
    ('classifier', LogisticRegression(penalty='l2'))
])

In [ ]:
param_grid = {
    'classifier__C': [0.01,0.5,0.99]
}

grid_search_03 = GridSearchCV(pipe, param_grid, scoring='roc_auc',cv=3)

In [ ]:
%%time
grid_search_03.fit(X_train,y_train)

In [ ]:
grid_search_03.best_score_

In [ ]:
y_pred = grid_search_03.predict(X_test)
y_pred_proba = grid_search_03.predict_proba(X_test)[:,1]
auc_res_03 = roc_auc_score(y_test,y_pred_proba)
auc_res_03

# Part_04: Подготавливаю результирующие файлы

In [ ]:
mod_res = grid_search_01
y_pred_res = mod_res.predict(test[categorical_columns + numerical_columns])
sub['isFraud'] = y_pred_res
sub.to_csv('Result_v01.csv', index=False)

In [ ]:
mod_res = grid_search_02
y_pred_res = mod_res.predict(test[categorical_columns + numerical_columns])
sub['isFraud'] = y_pred_res
sub.to_csv('Result_v02.csv', index=False)

In [ ]:
mod_res = grid_search_03
y_pred_res = mod_res.predict(test[categorical_columns + numerical_columns])
sub['isFraud'] = y_pred_res
sub.to_csv('Result_v03.csv', index=False)